Required packets

In [1]:
import torch
import pandas as pd
import numpy as np

Compare the original dataset with the tensor obtained after peek detection to find the intervall where all seasonality are present

In [ ]:
series = pd.read_csv('dataset/Custom/H_test_noise.csv', header=0, index_col=0, parse_dates=True)

column_names = series.columns.to_list()
time_index = series.index
wavelet_result = torch.load("result.pt")

rows_to_add = []

if len(column_names) > 1:
  # Itera su ciascuna riga del DataFrame
  for index, row in series.iterrows():
    tmp_row = [0] * len(row)
    row_index = series.index.get_loc(index)

    for i in range(0, len(column_names)):
      if torch.all(torch.eq(torch.tensor(1), wavelet_result[row_index, i])):
        tmp_row[i] = row[i]
      else:
        tmp_row[i] = np.nan
    rows_to_add.append(tmp_row)
else:
  for index, row in series.iterrows():
    tmp_row = [0] * len(row)
    row_index = series.index.get_loc(index)
    if torch.all(torch.eq(torch.tensor(1), wavelet_result[row_index])):
      tmp_row = row.values[0]
    else:
      tmp_row = np.nan
    rows_to_add.append(tmp_row)

# Create a DataFrame for the coefficients of the current dimension
final_df = pd.DataFrame(data=rows_to_add, index=time_index, columns=[f'{name}' for name in column_names])

final_df.to_csv('final_dataset.csv')